# --- Day 1: Sonar Sweep ---

### --- Part One ---

In [24]:
with open("day1-input1.txt", 'r') as file:
#     inputs = file.read()
    inputs = [int(line.replace('\n', '')) for line in file.readlines()]
    

In [25]:
inputs[-5:]

[8299, 8306, 8317, 8320, 8341]

In [26]:
counter = 0
for i in range(1, len(inputs)):
    if inputs[i] > inputs[i-1]:
        counter += 1

In [27]:
counter

1581

### --- Part Two ---

In [30]:
counter = 0
for i in range(1, len(inputs)-2):
    A = inputs[i-1] + inputs[i] + inputs[i+1]
    B = inputs[i] + inputs[i+1] + inputs[i+2]
    if B > A:
        counter += 1

In [31]:
counter

1618

# --- Day 2: Dive! ---

### --- Part One ---

In [33]:
with open("day2-input1.txt", 'r') as file:
#     inputs = file.read()
    inputs = [line.replace('\n', '') for line in file.readlines()]
    

In [40]:
depth = 0
horizontal = 0

for direction, v in [move.split() for move in inputs]:
    if direction == "forward":
        horizontal += int(v)
    elif direction == "down":
        depth += int(v)
    elif direction == "up":
        depth -= int(v)

In [42]:
depth * horizontal

1507611

### --- Part Two ---

In [ ]:
down X increases your aim by X units.
up X decreases your aim by X units.
forward X does two things:
It increases your horizontal position by X units.
It increases your depth by your aim multiplied by X.

In [45]:
depth = 0
horizontal = 0
aim = 0

for direction, v in [move.split() for move in inputs]:
    if direction == "forward":
        horizontal += int(v)
        depth += aim * int(v)
    elif direction == "down":
        aim += int(v)
    elif direction == "up":
        aim -= int(v)

In [46]:
depth * horizontal

1880593125

# --- Day 3: Binary Diagnostic ---

### --- Part One ---

In [3]:
with open("day3-input1.txt", 'r') as file:
#     inputs = file.read()
    inputs = [line.replace('\n', '') for line in file.readlines()]

In [11]:
cols_num = len(inputs[0])
rows_num = len(inputs)
print("Cols:", cols_num, "Rows:", rows_num)


Cols: 12 Rows: 1000


In [60]:
import numpy as np

diagnostic_data = np.asarray([int(cell) for row in inputs for cell in row]).reshape(1000,12)

In [61]:
diagnostic_data[:5]

array([[0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0]])

In [52]:
gamma = ''
for i in range(12):
    gamma += '1' if diagnostic_data[:,i].sum() > rows_num/2 else '0'
epsilon = ''.join(['0' if v == '1' else '1' for v in gamma]) 
print("  gamma: ", gamma)
print("epsilon: ", epsilon)

  gamma:  011100101100
epsilon:  100011010011


In [53]:
int(gamma, 2) * int(epsilon, 2)

4147524

### --- Part Two ---

In [175]:
def rating_calc(data, col=0, mode='oxygen'):
    if len(data) == 1:
        return data[0]
    # for oxygen:
    keep_val =  1 if data[:,col].sum() >= len(data)/2 else 0
    if mode == 'co2':
        keep_val =  1 - keep_val
    data = data[data[:,col] == keep_val]
    return rating_calc(data, col+1, mode)

In [176]:
oxygen_rating = rating_calc(diagnostic_data, mode='oxygen')
co2_rating = rating_calc(diagnostic_data, mode='co2')
print(oxygen_rating, co2_rating)

[0 1 0 1 1 0 0 1 0 0 1 1] [1 0 0 1 1 1 0 0 0 1 1 0]


In [177]:
int(''.join([str(v) for v in oxygen_rating]), 2) * int(''.join([str(v) for v in co2_rating]), 2) 

3570354

# --- Day 4: Giant Squid ---

### --- Part One ---

In [178]:
with open("day4-numbers.txt", 'r') as file:
    numbers = file.read()
#     numbers = [line.replace('\n', '') for line in file.readlines()]

In [180]:
numbers = [int(number) for number in numbers.split(',')]

In [313]:
with open("day4-boards.txt", 'r') as file:
    boards_raw = file.read().split('\n\n')

In [431]:
boards = [list(map(int, board.replace('\n', ' ').replace('  ', ' ').strip().split(' ')))
          for board in boards_raw]

In [432]:
boards_np = np.asarray(boards).reshape(-1,5,5)

In [433]:
def check_bingo(boards):
    pattern = np.asarray([True,True,True,True,True])
    horizontal = np.where((boards==pattern).all(axis=2))[0]
    vertical = np.where((boards==pattern.reshape(5,1)).all(axis=1))[0]
    return np.append(horizontal, vertical)

In [434]:
boards_np_01 = np.zeros_like(boards_np)
for i in numbers:
    boards_np_01 += boards_np == i
    bingo = check_bingo(boards_np_01)
    if len(bingo) > 0:
        print("BINGO!!!")
        break;

BINGO!!!


In [435]:
def calc_score(bingo, boards_np, boards_np_01, last_number):
    winning_board_01 = boards_np_01[bingo]
    winning_board_01_inv = np.logical_not(winning_board_01)
    winning_board = boards_np[bingo]
    final_score = winning_board[winning_board_01_inv].sum() * last_number
    return final_score

In [437]:
calc_score(bingo, boards_np, boards_np_01, i)

31424

### --- Part Two ---

In [438]:
boards_np = np.asarray(boards).reshape(-1,5,5)
boards_np_01 = np.zeros_like(boards_np)
for i in numbers:
    boards_np_01 += boards_np == i
    bingo = check_bingo(boards_np_01)
    if len(bingo) > 0:
        last_board_score = calc_score(bingo, boards_np, boards_np_01, i)
        # set values to something that will prevent these boards from further processing
        boards_np_01[bingo] = 0
        boards_np[bingo] = -1

In [439]:
last_board_score

23042